In [54]:
import graphlab as gl

In [55]:
Data = gl.SFrame('people_wiki.gl/')

In [56]:
Data.head()    #This data contains url  to the doc ,name of pearson doc related ,and test article about pearson

URI,name,text
<http://dbpedia.org/resource/Digby_Morrell> ...,Digby Morrell,digby morrell born 10october 1979 is a former ...
<http://dbpedia.org/resource/Alfred_J._Lewy> ...,Alfred J. Lewy,alfred j lewy aka sandylewy graduated from ...
<http://dbpedia.org/resource/Harpdog_Brown> ...,Harpdog Brown,harpdog brown is a singerand harmonica player who ...
<http://dbpedia.org/resource/Franz_Rottensteiner> ...,Franz Rottensteiner,franz rottensteiner bornin waidmannsfeld lower ...
<http://dbpedia.org/resource/G-Enka> ...,G-Enka,henry krvits born 30december 1974 in tallinn ...
<http://dbpedia.org/resource/Sam_Henderson> ...,Sam Henderson,sam henderson bornoctober 18 1969 is an ...
<http://dbpedia.org/resource/Aaron_LaCrate> ...,Aaron LaCrate,aaron lacrate is anamerican music producer ...
<http://dbpedia.org/resource/Trevor_Ferguson> ...,Trevor Ferguson,trevor ferguson aka johnfarrow born 11 november ...
<http://dbpedia.org/resource/Grant_Nelson> ...,Grant Nelson,grant nelson born 27april 1971 in london ...
<http://dbpedia.org/resource/Cathy_Caruth> ...,Cathy Caruth,cathy caruth born 1955 isfrank h t rhodes ...


In [57]:
len(Data)

59071

In [58]:
gl.canvas.set_target('browser')

In [59]:
obama=Data[Data['name']=='Barack Obama'] # we can use any name from looking into the data 
obama

URI,name,text
<http://dbpedia.org/resource/Barack_Obama> ...,Barack Obama,barack hussein obama iibrk husen bm born august ...


In [60]:
#create word count 
obama['wordcount']=gl.text_analytics.count_words(obama['text'])

In [61]:
obama_word_count_table=obama[['wordcount']].stack('wordcount',new_column_name=['word','count'])

In [62]:
obama_word_count_table.sort('count',ascending=False)


word,count
the,40
in,30
and,21
of,18
to,14
his,11
obama,9
act,8
he,7
a,7


In [63]:
Data['wordcount']=gl.text_analytics.count_words(Data['text'])

In [64]:
tfidf=gl.text_analytics.tf_idf(Data['wordcount'])

In [65]:
#tfidf

In [66]:
Data['tfidf'] = tfidf

In [67]:
#Data.head()

In [68]:
obama1=Data[Data['name']=='Barack Obama']

In [69]:
obama1[['tfidf']].stack('tfidf',new_column_name=['word','tfidf']).sort('tfidf',ascending=False)

word,tfidf
obama,43.2956530721
act,27.678222623
iraq,17.747378588
control,14.8870608452
law,14.7229357618
ordered,14.5333739509
military,13.1159327785
involvement,12.7843852412
response,12.7843852412
democratic,12.4106886973


# Manually compute distances between a few people

Let  manually compare the distances between the articles for a few famous people.



In [70]:
clinton = Data[Data['name'] == 'Bill Clinton']

In [71]:
beckham =Data[Data['name']== 'David Beckham']

Is Obama closer to Clinton than to Beckham............?

use cosine distance which is given by (1-cosine_similarity) 

In [72]:
obama1


Columns:
	URI	str
	name	str
	text	str
	wordcount	dict
	tfidf	dict

Rows: Unknown

Data:
+-------------------------------+--------------+-------------------------------+
|              URI              |     name     |              text             |
+-------------------------------+--------------+-------------------------------+
| <http://dbpedia.org/resour... | Barack Obama | barack hussein obama ii br... |
+-------------------------------+--------------+-------------------------------+
+-------------------------------+-------------------------------+
|           wordcount           |             tfidf             |
+-------------------------------+-------------------------------+
| {'operations': 1, 'represe... | {'operations': 3.811771079... |
+-------------------------------+-------------------------------+
[? rows x 5 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.

In [73]:
gl.distances.cosine(obama1['tfidf'][0],clinton['tfidf'][0])

0.8339854936884276

In [74]:
gl.distances.cosine(obama1['tfidf'][0],beckham['tfidf'][0])

0.9791305844747478

# create a nearest-neighbors model and apply it to document retrieval.  

In [75]:
knn_model = gl.nearest_neighbors.create(Data,features=['tfidf'],label='name')

Starting brute force nearest neighbors model training.

In [76]:
knn_model.query(obama1)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 24.28ms      |

| Done         |         | 100         | 616.311ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Barack Obama,0.0,1
0,Joe Biden,0.794117647059,2
0,Joe Lieberman,0.794685990338,3
0,Kelly Ayotte,0.811989100817,4
0,Bill Clinton,0.813852813853,5


# Explore some other celebrities .....

In [77]:
swift=Data[Data['name']== 'Taylor Swift']
angelina=Data[Data['name']== 'Angelina Jolie']
arnold=Data[Data['name']== 'Arnold Schwarzenegger']

In [78]:
knn_model.query(swift)


Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 28.597ms     |

| Done         |         | 100         | 619.134ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Taylor Swift,0.0,1
0,Carrie Underwood,0.76231884058,2
0,Alicia Keys,0.764705882353,3
0,Jordin Sparks,0.769633507853,4
0,Leona Lewis,0.776119402985,5


In [79]:
knn_model.query(angelina)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 22.504ms     |

| Done         |         | 100         | 633.808ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Angelina Jolie,0.0,1
0,Brad Pitt,0.784023668639,2
0,Julianne Moore,0.795857988166,3
0,Billy Bob Thornton,0.803069053708,4
0,George Clooney,0.8046875,5


In [80]:
knn_model.query(arnold)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 24.535ms     |

| Done         |         | 100         | 635.501ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Arnold Schwarzenegger,0.0,1
0,Jesse Ventura,0.818918918919,2
0,John Kitzhaber,0.824615384615,3
0,Lincoln Chafee,0.833876221498,4
0,Anthony Foxx,0.833910034602,5
